In [7]:
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plotting.plot as rbsplot
import rba_tools.backtest.backtrader_extensions.plotting.plotinfo as rbsplotinfo
import importlib
import rba_tools.backtest.backtrader_extensions.strategies as rbsstrat

In [3]:
importlib.reload(rbsplot)

<module 'rba_tools.backtest.backtrader_extensions.plot' from 'c:\\users\\avery\\documents\\github\\rba_tools_clone\\src\\rba_tools\\backtest\\backtrader_extensions\\plot.py'>

In [3]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [4]:
cerebro = bt.Cerebro(runonce=False)

# Add a strategy
#cerebro.addstrategy(MaCrossStrategy)
cerebro.addstrategy(rbsstrat.TestStrategy, period=7)
#cerebro.addstrategy(rbsstrat.ConsecutiveBarsTest)
#cerebro.addstrategy(rbsstrat.StopLimitEntryStrategy)

#get a datapuller
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()

#symbol and date range
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '6/30/20'

# Get a pandas dataframe
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)

# Pass it to the backtrader datafeed and add it to the cerebro
data = bt.feeds.PandasData(dataname=dataframe,
                            nocase=True,
                            )
#cerebro.adddata(data)
cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=240)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=1440)

# Run over everything
back = cerebro.run()

# Plot the result
#myp = cerebro.plot(numfigs=1, style='bar')

In [5]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [8]:
dpi = rbsplotinfo.DataAndPlotInfoContainer(strategy)
dpi

DataAndPlotInfoContainer(strategy='TestStrategy', data_and_plots_list=[DataPlotInfo(df=                       Open    High     Low   Close        Volume  \
2020-01-01 00:00:00  128.66  128.66  128.19  128.42    515.780816   
2020-01-01 04:00:00  128.33  130.58  128.30  129.67   1857.032786   
2020-01-01 08:00:00  129.67  130.25  129.60  129.81   1191.872012   
2020-01-01 12:00:00  130.03  131.50  129.95  131.32    939.921635   
2020-01-01 16:00:00  131.31  132.79  131.01  132.68   2201.396240   
...                     ...     ...     ...     ...           ...   
2020-06-30 08:00:00  226.39  227.15  225.77  226.05   4074.983230   
2020-06-30 12:00:00  226.01  228.01  223.51  224.76   8255.857590   
2020-06-30 16:00:00  224.52  226.73  222.80  225.75  12605.506344   
2020-06-30 20:00:00  225.86  225.97  224.51  225.47   2980.591000   
2020-07-01 00:00:00  225.58  226.46  225.01  225.68   1415.758444   

                     cash (fund=None)  value (fund=None)   (usenames=True)  \
2020-0

In [9]:
for ind in dpi.data_and_plots_list[0].indicator_list:
    rbsplot.plot_indicator(dpi.data_and_plots_list[0].df, ind).show()

In [16]:
dpi.df_list[0]

,Open,High,Low,Close,Volume,cash (fund=None),value (fund=None),(usenames=True),buy (barplot=True bardist=0.015),sell (barplot=True bardist=0.015),consec_bars (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_started_ago (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_high (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_retrace (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_open (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85)
2020-01-01 00:00:00,128.66,128.66,128.19,128.42,515.780816,10000.000,10000.000,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2020-01-01 04:00:00,128.33,130.58,128.30,129.67,1857.032786,10000.000,10000.000,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2020-01-01 08:00:00,129.67,130.25,129.60,129.81,1191.872012,10000.000,10000.000,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2020-01-01 12:00:00,130.03,131.50,129.95,131.32,939.921635,10000.000,10000.000,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
2020-01-01 16:00:00,131.31,132.79,131.01,132.68,2201.396240,10000.000,10000.000,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-30 08:00:00,226.39,227.15,225.77,226.05,4074.983230,9913.355,9913.355,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
2020-06-30 12:00:00,226.01,228.01,223.51,224.76,8255.857590,9913.355,9913.355,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
2020-06-30 16:00:00,224.52,226.73,222.80,225.75,12605.506344,9913.355,9913.355,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
2020-06-30 20:00:00,225.86,225.97,224.51,225.47,2980.591000,9913.355,9913.355,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN


In [17]:
dpi.df_list[1]

,Open,High,Low,Close,Volume,buy (barplot=True bardist=0.015),sell (barplot=True bardist=0.015),trend_high (),trend_open (),consec_bars (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_started_ago (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_high (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_retrace (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_open (minimum_bars=2 maximum_bars=20 minimum_move_percent=10 trend_retrace_percent=85),trend_retrace ()
2020-01-01,128.66,128.66,128.19,128.42,515.780816,NaN,NaN,NaN,NaN,-6.0,NaN,NaN,NaN,NaN,NaN
2020-01-02,128.33,132.86,128.30,130.19,13163.487459,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2020-01-03,130.06,130.06,126.00,126.71,26250.065941,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN
2020-01-04,126.69,134.72,125.40,133.58,50313.895117,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2020-01-05,133.36,135.91,132.58,134.78,18000.273569,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-27,232.93,233.19,226.86,229.30,41971.814456,NaN,NaN,NaN,NaN,-3.0,NaN,NaN,NaN,NaN,NaN
2020-06-28,229.51,230.89,215.90,219.35,60857.542028,NaN,NaN,NaN,NaN,-4.0,NaN,NaN,NaN,NaN,NaN
2020-06-29,219.35,227.57,217.80,224.61,38486.237540,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2020-06-30,224.63,229.82,221.19,227.76,43663.927594,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
